In [18]:
import folium
#geopy imports 
from geopy.extra.rate_limiter import RateLimiter
from geopy import Nominatim

#geocode for physical address to lat/long lookup
locator = Nominatim(user_agent="myGeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
pd.set_option('display.max_rows', 200)

In [2]:
vets = pd.read_csv('vets_mapped.csv',  encoding="ISO-8859-1")
data = vets['Combined'].value_counts()

In [11]:
data = data.to_frame()
data['Location'] = data.index
data.index = range(len(data))

#Drop 'USA' entries (they plot in the middle of Kansas) and reindex to length of data
data.drop(2, inplace=True)
data.index = range(len(data))

In [15]:
data.drop(188, inplace=True)

In [16]:
data['Location'] = data['Location'].str.split(',').apply(lambda x: x[0]) + ',' + data['Location'].str.split(',').apply(lambda x: x[1]).str.replace('CA', "California")

In [19]:
data['Coordinates'] = data['Location'].apply(geocode)

In [21]:
#convert coordinates to individual lat and longs
data['point'] = data['Coordinates'].apply(lambda loc: tuple(loc.point) if loc else None)
data[['latitude', 'longitude', 'altitude']] = pd.DataFrame(data['point'].tolist(), index=data.index)

In [24]:
data.drop(['point', 'altitude'], axis=1, inplace=True)

In [38]:
data.head()

,Combined,Location,Coordinates,latitude,longitude
0,92,"San Diego, California","(San Diego, San Diego County, California, Unit...",32.717421,-117.162771
1,79,"Virginia Beach, VA","(Virginia Beach, Virginia Beach (city), Virgin...",36.852984,-75.977418
2,29,"Washington, DC","(Washington, District of Columbia, 20230, Unit...",38.894992,-77.036558
3,26,"New York, NY","(New York, United States of America, (40.71272...",40.712728,-74.006015
4,24,"Fayetteville, NC","(Fayetteville, Cumberland County, North Caroli...",35.052576,-78.878292


In [40]:
data.to_csv('Mapping/vets_coords.csv', index=False)

In [26]:
#base coordinates
USA = (39.8283, 98.5795)
 
# for speed purposes
MAX_RECORDS = 1000
  
# create empty map zoomed in on geographic center of US
m = folium.Map(location=USA, zoom_start=5)

In [ ]:
# add a marker for every record in the filtered data, use a clustered view
for each in crimedata[0:MAX_RECORDS].iterrows():
    map.simple_marker(
        location = [each[1]['Y'],each[1]['X']], 
        clustered_marker = True)
  
display(map)

In [37]:
import plotly.graph_objects as go

import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2011_february_us_airport_traffic.csv')
df['text'] = df['airport'] + '' + df['city'] + ', ' + df['state'] + '' + 'Arrivals: ' + df['cnt'].astype(str)


fig = go.Figure(data=go.Scattergeo(
        locationmode = 'USA-states',
        lon = df['long'],
        lat = df['lat'],
        text = df['text'],
        mode = 'markers',
        marker = dict(
            size = 8,
            opacity = 0.8,
            reversescale = True,
            autocolorscale = False,
            symbol = 'square',
            line = dict(
                width=1,
                color='rgba(102, 102, 102)'
            ),
            colorscale = 'Blues',
            cmin = 0,
            color = df['cnt'],
            cmax = df['cnt'].max(),
            colorbar_title="Incoming flights<br>February 2011"
        )))

fig.update_layout(
        title = 'Most trafficked US airports<br>(Hover for airport names)',
        geo = dict(
            scope='usa',
            projection_type='albers usa',
            showland = True,
            landcolor = "rgb(250, 250, 250)",
            subunitcolor = "rgb(217, 217, 217)",
            countrycolor = "rgb(217, 217, 217)",
            countrywidth = 0.5,
            subunitwidth = 0.5
        ),
    )
fig.show()

ModuleNotFoundError: No module named 'plotly'